In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

import sys
sys.path.append('../../')

import os
import copy
import pickle
from datetime import datetime, date, timedelta

from utils.generic.config import read_config, make_date_key_str
from utils.generic.reichlab import *
from viz.reichlab import *
from viz import plot_ptiles
from viz.uncertainty import plot_ptiles_reichlab

In [ ]:
predictions_pkl_filename = '/scratch/users/sansiddh/covid-modelling/2020_1206_120051/predictions_dict.pkl'
with open(predictions_pkl_filename, 'rb') as f:
    predictions_dict_d = pickle.load(f)

In [ ]:
predictions_pkl_filename = '/scratch/users/sansiddh/covid-modelling/2020_1206_120100/predictions_dict.pkl'
with open(predictions_pkl_filename, 'rb') as f:
    predictions_dict_t = pickle.load(f)

In [ ]:
loc_name_to_key_dict = get_mapping(which='location_name_to_code')
us_states_abbv_dict = get_mapping(which='location_name_to_abbv')

In [ ]:
config = predictions_dict[list(predictions_dict.keys())[0]]['m2']['run_params']
loss_comp = config['loss']['loss_compartments'][0]
data_last_date = config['split']['end_date']
date_of_submission = (data_last_date + timedelta(days=2)).strftime('%Y-%m-%d')
if loss_comp == 'deceased':
    comp = 'cum_death'
if loss_comp == 'total':
    comp = 'cum_case'
print(comp)
print(date_of_submission)

In [ ]:
loc_name_to_key_dict

In [ ]:
df_wiai_submission_death = format_wiai_submission(predictions_dict_d, loc_name_to_key_dict, which_fit='m2', formatting_mode='submission',                                
                                                  use_as_point_forecast='ensemble_mean', skip_percentiles=False)

In [ ]:
df_wiai_submission_cases = format_wiai_submission(predictions_dict_t, loc_name_to_key_dict, which_fit='m2', formatting_mode='submission', 
                                                  use_as_point_forecast='ensemble_mean', skip_percentiles=False)

In [ ]:
df_wiai_submission_death = df_wiai_submission_death[df_wiai_submission_death['target'].apply(lambda x: 'death' in x)]
df_wiai_submission_cases = df_wiai_submission_cases[df_wiai_submission_cases['target'].apply(lambda x: 'inc case' in x)]

In [ ]:
df_wiai_submission_cases = df_wiai_submission_cases[np.logical_or(
    (df_wiai_submission_cases['type'] == 'point'), 
    (df_wiai_submission_cases['quantile'].isin([0.025, 0.100, 0.250, 0.500, 0.750, 0.900, 0.975])))]

In [ ]:
df_wiai_submission_death.reset_index(inplace=True, drop=True)
df_wiai_submission_cases.reset_index(inplace=True, drop=True)

In [ ]:
df_wiai_submission_comb = pd.concat([df_wiai_submission_death, df_wiai_submission_cases], ignore_index=True)
df_wiai_submission_comb

In [ ]:
places_to_prune = ['Guam', 'Northern Mariana Islands', 'Virgin Islands']
places_to_prune_code = [loc_name_to_key_dict[x] for x in places_to_prune]

In [ ]:
df_wiai_submission_comb = df_wiai_submission_comb[np.logical_not(df_wiai_submission_comb['location'].isin(places_to_prune_code))]
df_wiai_submission_comb.reset_index(inplace=True, drop=True)

In [ ]:
del df_wiai_submission_comb['model']

In [ ]:
df_wiai_submission_comb

In [ ]:
forecast_date = df_wiai_submission_comb.loc[0, 'forecast_date'].strftime('%Y-%m-%d')
model_name = 'Wadhwani_AI-BayesOpt'

In [ ]:
df_wiai_submission_comb.to_csv(f'../../../covid19-forecast-hub/data-processed/{model_name}/{forecast_date}-{model_name}.csv', index=False)